In [139]:
from sklearn import cross_validation
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
from time import time

In [140]:
execfile('ML_Challenge_data_preprocessing.py')

('Generated X with shape ', (347698, 1000))
('Generated y with shape ', (347698,))


In [141]:
bigrams = []
with open('possible_bigrams.csv', 'rb') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        bigrams = row

bigrams.sort()
bigrams_dict = {key: i for i, key in enumerate(bigrams)}
bigrams_dict

{'AA': 0,
 'AC': 1,
 'AG': 2,
 'AT': 3,
 'CA': 4,
 'CC': 5,
 'CG': 6,
 'CT': 7,
 'GA': 8,
 'GC': 9,
 'GG': 10,
 'GT': 11,
 'TA': 12,
 'TC': 13,
 'TG': 14,
 'TT': 15}

In [142]:
X2 = []
y2 = []
for i, x in enumerate(X):
    if 'N' not in X[i,:]:
        X2.append(x)
        y2.append(y[i])
        
X2 = np.array(X2)
y2 = np.array(y2)
print X2.shape
print y2.shape

(347683, 1000)
(347683,)


In [145]:
execfile('ngram_features.py')
# for every observation, create a new feature vector or length len(bigrams_dict.keys())

sample_size = 20000 #X.shape[0])
start = time()
newX = np.zeros((sample_size,len(bigrams_dict)))

for i in range(sample_size):
    newX[i,:] = ngram_features(X2[i,:], 2, bigrams_dict)
finish = time()
print 'time spent creating new features for {0} observations: {1}'.format(sample_size, finish - start)
# will take about 11 minutes for the whole data set
print newX.shape
print newX[0,:]

time spent creating new features for 20000 observations: 67.6003491879
(20000, 16)
[  15.   31.   59.   14.   36.  160.  127.   57.   58.  156.  150.   25.
   10.   34.   52.   15.]


In [146]:
test_size = 0.1
y_sample = y[0:sample_size]
X_train, X_test, y_train, y_test = cross_validation.train_test_split(
    newX, y_sample, test_size=test_size, random_state=0)
print X_train.shape
print y_train.shape

(18000, 16)
(18000,)


In [147]:
from sklearn.ensemble import RandomForestClassifier
from time import time

start_time = time()
clf = RandomForestClassifier(n_estimators=200, criterion='gini', max_features=4)
print clf.fit(X_train, y_train)
end_time = time()

print 'time spent to train ' + str(1 - test_size) + ': ' + str(end_time - start_time) + ' seconds.'

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=4, max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
time spent to train 0.9: 9.19432878494 seconds.


In [148]:
clf.score(X_test, y_test)

0.90649999999999997

In [149]:
from operator import itemgetter
from scipy.stats import randint as sp_randint
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from time import time

clf = RandomForestClassifier()


# Utility function to report best scores
def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")


# specify parameters and distributions to sample from
param_dist = {"n_estimators": sp_randint(1, 200),
              "max_depth": [5, 3, None],
              "max_features": sp_randint(1, 11),
              "min_samples_split": sp_randint(1, 11),
              "min_samples_leaf": sp_randint(1, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run randomized search
n_iter_search = 100
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search)

start = time()
random_search.fit(X_train, y_train)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.grid_scores_)

RandomizedSearchCV took 591.93 seconds for 100 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.894 (std: 0.003)
Parameters: {'bootstrap': False, 'min_samples_leaf': 3, 'n_estimators': 79, 'min_samples_split': 6, 'criterion': 'entropy', 'max_features': 5, 'max_depth': None}

Model with rank: 2
Mean validation score: 0.893 (std: 0.004)
Parameters: {'bootstrap': False, 'min_samples_leaf': 5, 'n_estimators': 72, 'min_samples_split': 6, 'criterion': 'entropy', 'max_features': 5, 'max_depth': None}

Model with rank: 3
Mean validation score: 0.893 (std: 0.004)
Parameters: {'bootstrap': True, 'min_samples_leaf': 4, 'n_estimators': 151, 'min_samples_split': 2, 'criterion': 'gini', 'max_features': 6, 'max_depth': None}

